In [1]:
import robosandbox as rsb
import numpy as np
import plotly.graph_objects as go
from tqdm import tqdm
from robosandbox.performance.WorkSpace import WorkSpace

In [2]:
def evaluate_alpha_effect(
    alpha_list1, alpha_list2, isSave=False, method="invcondition", axes="all"
):
    """
    Evaluate the effect of alpha on the global indices.
    """
    results = []
    res_mat = np.zeros((len(alpha_list1), len(alpha_list2)))
    for i, alpha1 in tqdm(
        enumerate(alpha_list1),
        total=len(alpha_list1),
        desc="Alpha1 progress",
        unit="outer",
    ):
        for j, alpha2 in tqdm(
            enumerate(alpha_list2),
            total=len(alpha_list2),
            desc=f"Alpha2 progress (alpha1={np.rad2deg(alpha1)})",
            unit="inner",
            leave=False,
        ):
            robot = rsb.models.DH.Generic.GenericFour(alpha=[np.pi / 2, alpha1, alpha2, 0])
            ws = WorkSpace(robot=robot)
            G = ws.iter_calc_global_indice(
                    initial_samples=3000,
                    batch_ratio=0.1,
                    error_tolerance_percentage=1e-3,
                    method="invcondition",
                    axes="all",
                    max_samples=30000,
                )
            results.append([alpha1, alpha2, G])
            res_mat[i, j] = G
            del robot, ws

    results = np.array(results)
    if isSave:
        filename = f"data/two_alpha/{method}_{axes}.npz"
        np.savez(filename, results=results, res_mat=res_mat)
    return results, res_mat


In [3]:
def plot_global_index_results(
    alpha_list_deg1,
    alpha_list_deg2,
    res_mat,
    plot_type="heatmap",
    method="invcondition",
    axes="all",
    isSave=False,
):
    """
    Plot the effect of alpha on the global indices using Plotly.
    """
    G_mat = res_mat
    print(f"alpha 1: {alpha_list_deg1}")
    print(f"alpha 2: {alpha_list_deg2}")
    # print(G_mat)
    fig = go.Figure()
    if plot_type == "heatmap":
        fig = go.Figure(
            data=go.Heatmap(
                z=G_mat,
                x=alpha_list_deg1,
                y=alpha_list_deg2,
                # colorscale='Viridis',
                colorbar=dict(
                    title=f"{method}", titlefont=dict(size=40), tickfont=dict(size=40)
                ),
            )
        )

    if plot_type == "surface":
        fig = go.Figure(
            data=go.Surface(
                z=G_mat,
                x=alpha_list_deg1,
                y=alpha_list_deg2,
                colorbar=dict(
                    title=f"{method}", titlefont=dict(size=40), tickfont=dict(size=40)
                ),
            )
        )
        fig.update_layout(
            scene=dict(
                xaxis_title="alpha1 (deg)",
                yaxis_title="alpha2 (deg)",
                zaxis_title="",
                xaxis=dict(titlefont=dict(size=40), tickfont=dict(size=16), dtick=90),
                yaxis=dict(titlefont=dict(size=40), tickfont=dict(size=16), dtick=90),
                zaxis=dict(titlefont=dict(size=40), tickfont=dict(size=16), dtick=90),
                camera=dict(
                    eye=dict(x=1.55, y=1.55, z=1.55),
                    up=dict(x=0, y=0, z=1),
                ),
            )
        )

    fontsize = 40
    fig.update_layout(
        # title=f'Effect of alpha on global indices using {method} method and {axes} axes',
        xaxis_title="alpha1 (deg)",
        yaxis_title="alpha2 (deg)",
        autosize=True,
        height=800,
        width=1000,
        xaxis_title_font=dict(size=40),  # Font size for x-axis title
        yaxis_title_font=dict(size=40),
        xaxis=dict(tickfont=dict(size=fontsize), dtick=30),
        yaxis=dict(tickfont=dict(size=fontsize), dtick=30),
    )

    if isSave:
        fig.write_image(f"fig/two_alpha/{method}_{axes}_{plot_type}.png")
        fig.write_html(f"fig/two_alpha/{method}_{axes}_{plot_type}.html")
    fig.show()

In [6]:
np.random.seed(42)

# from 0, 10, 20, ..., 180 deg for alpha
alpha_list_deg1 = np.arange(0, 181, 15)
alpha_list_rad1 = np.deg2rad(alpha_list_deg1)
alpha_list_deg2 = np.arange(0, 181, 15)
alpha_list_rad2 = np.deg2rad(alpha_list_deg2)
print(f"alpha 1: {alpha_list_deg1}")
print(f"alpha 2: {alpha_list_deg2}")

# method = "yoshikawa"
method = "invcondition"
axes = "all"

isRun = False
isSave = True
# isSave = False
# isRun = False
if isRun:
    res, res_mat = evaluate_alpha_effect(
        alpha_list_rad1, alpha_list_rad2, isSave=True, method=method, axes=axes
    )
if not isRun:
    npzfile = np.load(f"data/two_alpha/{method}_{axes}.npz")
    res = npzfile["results"]
    res_mat = npzfile["res_mat"]
plot_global_index_results(
    alpha_list_deg1,
    alpha_list_deg2,
    res_mat,
    plot_type="surface",
    method=method,
    axes=axes,
    isSave=isSave,
)

alpha 1: [  0  15  30  45  60  75  90 105 120 135 150 165 180]
alpha 2: [  0  15  30  45  60  75  90 105 120 135 150 165 180]
alpha 1: [  0  15  30  45  60  75  90 105 120 135 150 165 180]
alpha 2: [  0  15  30  45  60  75  90 105 120 135 150 165 180]


In [5]:
robot = rsb.models.DH.Generic.GenericFour(alpha=[np.pi / 2, 0, 0, 0])
ws = WorkSpace(robot=robot)
G = ws.iter_calc_global_indice(
        initial_samples=3000,
        batch_ratio=0.1,
        error_tolerance_percentage=1e-3,
        method="invcondition",
        axes="all",
        max_samples=30000,
    )

==> Iteration finished.
Converged after 2 iterations.
Final global indice: 0.028093630891616855
Final number of samples: 3300
Final relative error: 0.0005111925275763338
